# Silver Layer Analysis

`This notebook performs exploratory and validation analysis on the Silver datasets
to ensure data consistency and analytical readiness.`


In [0]:
silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/Silver"


In [0]:
display(dbutils.fs.ls(silver_path))

In [0]:
# loan_table 
loan_df = spark.read.format("delta").load(f"{silver_path}/loan_enriched")
display(loan_df)


### Portfolio Structure by Product

`This analysis aims to understand how the credit portfolio is distributed across product types.`\
`In credit risk frameworks such as IFRS 9, PD, LGD, and EAD models are systematically segmented by product category, as each product exhibits distinct risk characteristics.`\
`Analyzing the portfolio structure by product is therefore a necessary first step to interpret risk metrics and to provide a meaningful view of the portfolio composition.`


In [0]:

df_prod = (
    loan_df
        .groupBy("product_type")
        .agg(
            F.count("*").alias("nb_loans"),
            F.sum("principal_amount").alias("total_exposure"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("total_exposure", ascending=False)
)

display(df_prod)


product_type,nb_loans,total_exposure,default_rate
IMMO,37930,1.3243416017832453E10,1.0
CONSO,96489,2.0322709945902796E9,1.0
REVOLVING,1505,8215390.96650434,1.0


In [0]:
# There is a problem with the variable default_flag 
loan_df.select("default_flag").distinct().show()

+------------+
|default_flag|
+------------+
|        NULL|
|           1|
+------------+



In [0]:

# We assume that default events have been joined without explicitly encoding the absence of an event.
# Missing default indicators are therefore interpreted as non-default cases and are replaced with zero before proceeding with the analysis.


loan_df = loan_df.withColumn(
    "default_flag",
    F.when(F.col("default_flag").isNull(), 0).otherwise(F.col("default_flag")))


In [0]:
loan_df.select("default_flag").distinct().show()

+------------+
|default_flag|
+------------+
|           1|
|           0|
+------------+



In [0]:

df_prod = (
    loan_df
        .groupBy("product_type")
        .agg(
            F.count("*").alias("nb_loans"),
            F.sum("principal_amount").alias("total_exposure"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("total_exposure", ascending=False)
)

display(df_prod)


product_type,nb_loans,total_exposure,default_rate
IMMO,37930,1.3243416017832453E10,0.02842077511204851
CONSO,96489,2.0322709945902796E9,0.0734384230326773
REVOLVING,1505,8215390.96650434,0.12691029900332226


##### Interpretation of Results

`The observed default rates by product type are consistent with typical retail banking risk profiles.`\
`Mortgage loans exhibit a low default rate (around 2.8%), which is realistic given the presence of strong collateral, more solvent borrowers, larger loan amounts, and generally more stable repayment capacity.`\
`Consumer loans show a higher default rate (around 7.3%), reflecting their unsecured nature, easier access, and their frequent use by more financially constrained borrowers.`\
`Revolving credit displays the highest default rate (around 12.7%), which is expected for this product category.`\
`Revolving credit is structurally riskier due to the absence of collateral, high utilization behavior, elevated interest rates, fragile borrower profiles, debt cycles, and high credit conversion factors (CCF) reflecting frequent usage of credit limits.`



### Customer Risk Analysis — Default Rate by Income Segment

`The objective of this analysis is to assess whether lower-income customers carry higher credit risk.`

`Annual income is a key driver of repayment capacity and is therefore a fundamental variable in IFRS 9 frameworks and PD modeling.`

`Analyzing default rates by income segment allows the identification of more fragile customer groups, whose solvency and income stability may be more sensitive to economic conditions.`

`Customers are segmented into low, medium, and high income buckets to facilitate comparative risk analysis.`


In [0]:
loan_df.select("annual_income").show()

+------------------+
|     annual_income|
+------------------+
| 45057.86837163758|
| 82359.76886963609|
|45276.664880487006|
|63915.201074141914|
|  21400.2783605274|
|28312.678256321517|
| 32868.53091173815|
| 22335.39213692309|
| 35595.03540649403|
| 40590.42106010997|
|24079.792073740893|
| 52627.82106424119|
| 73093.51123168226|
| 26210.94008856016|
|37861.880721824375|
| 26950.23062394332|
| 72935.34230545466|
| 23087.01074096215|
|54594.420105640456|
|55031.774173592195|
+------------------+
only showing top 20 rows



In [0]:
loan_df= loan_df.withColumn(
    "income_bucket",
    F.when(F.col("annual_income") < 20000, "LOW")
     .when((F.col("annual_income") >= 20000) & (F.col("annual_income") < 40000), "MEDIUM")
     .otherwise("HIGH")
)


In [0]:
df_income = (
    loan_df
        .groupBy("income_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.sum("principal_amount").alias("exposure"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("income_bucket")
)

display(df_income)


income_bucket,nb_loans,exposure,default_rate
HIGH,83470,9.367762683126446E9,0.05357613513837307
LOW,4311,4.808110819972477E8,0.1057759220598469
MEDIUM,48143,5.43532863826556E9,0.07118376503333818


##### Interpretation by Income Segment

`The LOW income segment exhibits the highest default rate (approximately 10.6%), indicating a structurally fragile borrower profile.`\
`Low-income customers typically have limited repayment capacity, high sensitivity to economic shocks, more unstable employment, frequent use of consumer and revolving credit, and little financial buffer.`\
`Although this segment represents a relatively small share of the portfolio in volume, it carries disproportionate risk and is therefore critical in PD analysis.`\
`The MEDIUM income segment shows an intermediate level of risk, with a default rate around 7.1%, reflecting more balanced but still vulnerable borrower profiles.`\
`The HIGH income segment is both the largest and the safest, with the lowest default rate (around 5.3%).`\
`High-income customers concentrate the majority of the exposure while exhibiting the strongest repayment capacity and the lowest observed credit risk.`


### Credit Score Analysis

`The objective of this analysis is to assess the relationship between credit score and default risk.`\
`Credit score is a core indicator of borrower creditworthiness and is a central input in PD modeling and IFRS 9 risk frameworks.`\
`To facilitate analysis and interpretation, borrowers are grouped into discrete credit score buckets before evaluating default behavior across segments.`


In [0]:
loan_df = loan_df.withColumn(
    "credit_score_bucket",
    F.when(F.col("credit_score") < 500, "LOW")
     .when((F.col("credit_score") >= 500) & (F.col("credit_score") < 650), "MEDIUM")
     .otherwise("HIGH")
)
 

In [0]:
df_score = (
    loan_df
        .groupBy("credit_score_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.sum("principal_amount").alias("exposure"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("credit_score_bucket")
)

display(df_score)


credit_score_bucket,nb_loans,exposure,default_rate
HIGH,93357,1.0469064301844631E10,0.04794498537870754
LOW,3533,4.065085652030793E8,0.13274837248797056
MEDIUM,39034,4.408329536341547E9,0.08735973766460009



##### Interpretation of Results 

`Default probabilities decrease as credit score increases, from approximately 13% for low-score customers to around 5% for high-score customers.`\
`This behavior is fully consistent with standard banking practices and confirms the quality of the portfolio segmentation.`


### Analysis by Age Segment

`Borrowers are grouped into five age segments to analyze default behavior across life stages.`\
`Default rates are computed for each age segment to assess how credit risk evolves with borrower age.`


In [0]:

loan_df = loan_df.withColumn(
  "age_bucket",
  F.when(F.col("age")< 26, "18-25")
   .when((F.col("age") >= 26) & (F.col("age") < 41), "26-40")
   .when((F.col("age") >= 41) & (F.col("age") < 61), "41-60")
  .otherwise("60+")
)

In [0]:
df_age = (
    loan_df
        .groupBy("age_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.sum("principal_amount").alias("exposure"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("age_bucket")
)

display(df_age)

age_bucket,nb_loans,exposure,default_rate
18-25,14043,1.580531228275617E9,0.059602649006622516
26-40,41490,4.626941610363953E9,0.061653410460351894
41-60,55330,6.267245068422494E9,0.061792879089101756
60+,25061,2.8091844963272066E9,0.06148996448665257


##### Interpretation of Results 

`The analysis by age segment shows a relatively stable default rate around 6% across all age groups.`\
`Age does not appear to be a major discriminating factor of credit risk in this simulated portfolio, which is consistent with a framework where credit score, income, and product type are the primary risk drivers.`

### Cross-Analysis: Product × Income - A cross-risk perspective

`This type of cross-analysis is systematically used in banking, as it combines product risk, customer risk, portfolio fragility, and profitability.`\
`It provides a consolidated view of where risk is concentrated and is typically one of the first analyses reviewed by risk management teams.This analysis is essential because:`

`A high-risk product sold to a fragile customer represents a significant risk.` 
`A high-risk product sold to a solid customer can be profitable.`  
`A low-risk product sold to a fragile customer is generally acceptable.`  
`A low-risk product sold to a solid customer represents the optimal risk-return profile.`


In [0]:
df_cross = (
    loan_df
        .groupBy("product_type", "income_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("product_type", "income_bucket")
)

display(df_cross)

product_type,income_bucket,nb_loans,default_rate
CONSO,HIGH,59368,0.06399070206171675
CONSO,LOW,3068,0.12777053455019557
CONSO,MEDIUM,34053,0.0850145361642146
IMMO,HIGH,23202,0.024480648219981038
IMMO,LOW,1194,0.04522613065326633
IMMO,MEDIUM,13534,0.03369292153095907
REVOLVING,HIGH,900,0.11666666666666667
REVOLVING,LOW,49,0.20408163265306123
REVOLVING,MEDIUM,556,0.1366906474820144


`The cross-analysis of product type and income reveals risk patterns that are fully consistent with typical banking portfolios`\
`Low-income customers systematically exhibit the highest default rates across all product categories.`\
`Mortgage products remain the most stable, with default rates typically between 2% and 4%, even within low-income segments.`\
`Revolving credit concentrates the highest risk, with default rates exceeding 20% in the low-income segment.`\
`These results confirm that income is a major driver of credit risk and that product characteristics significantly amplify or mitigate this sensitivity.`



### Vintage Analysis — Default Rate by Origination Year

`This analysis examines default rates by loan origination year to assess the evolution of credit risk across vintages.`\
`Vintage analysis helps identify changes in underwriting quality, portfolio risk appetite, and macroeconomic effects over time`


In [0]:

loan_df = loan_df.withColumn(
    "vintage_year",
    F.year(F.col("origination_date"))
) 

In [0]:
df_vintage = (
    loan_df
        .groupBy("vintage_year")
        .agg(
            F.count("*").alias("nb_loans"),
            F.sum("principal_amount").alias("exposure"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("vintage_year")
)

display(df_vintage)

vintage_year,nb_loans,exposure,default_rate
2018,19441,2.1843134328183603E9,0.06157090684635564
2019,19425,2.1757958828671E9,0.06383526383526383
2020,19402,2.209033283798316E9,0.06097309555715905
2021,19596,2.1841830063385305E9,0.061543172075933864
2022,19483,2.197313542245272E9,0.06148950366986604
2023,19220,2.1625231476905236E9,0.05967741935483871
2024,19357,2.170740107631158E9,0.061166503073823424


##### Interpretation of Results 

`The analysis by origination year shows a remarkable stability of credit risk over time.`\
`Default rates remain within a narrow range (approximately 5.9%–6.3%), with no evidence of structural deterioration in the portfolio.`\
`More recent vintages (2023–2024) naturally exhibit lower default rates due to their shorter seasoning period.`\
`Overall, the results indicate a healthy and consistent portfolio, with no signs of abnormal credit policy shifts or significant macroeconomic shocks.`


### Cross-Analysis — Credit Score × Income

`This analysis examines the combined effect of credit score and income on default risk.`\
`By crossing these two key borrower dimensions, it highlights how repayment capacity and creditworthiness jointly shape credit risk levels.`


In [0]:
df_score_income = (
    loan_df
        .groupBy("credit_score_bucket", "income_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("credit_score_bucket", "income_bucket")
)

display(df_score_income)


credit_score_bucket,income_bucket,nb_loans,default_rate
HIGH,HIGH,69529,0.047979979576867206
HIGH,LOW,1,0.0
HIGH,MEDIUM,23827,0.04784488185671717
LOW,LOW,708,0.12429378531073447
LOW,MEDIUM,2825,0.13486725663716814
MEDIUM,HIGH,13941,0.08148626353920092
MEDIUM,LOW,3602,0.10216546363131594
MEDIUM,MEDIUM,21491,0.08868828812060862



### Cross-Analysis — Credit Score × Age

`This analysis explores the interaction between credit score and borrower age in explaining default risk.`\
`It helps assess whether creditworthiness dominates age effects or whether specific life-stage patterns emerge in the portfolio.`


In [0]:
df_score_age = (
    loan_df
        .groupBy("credit_score_bucket", "age_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("credit_score_bucket", "age_bucket")
)

display(df_score_age)




credit_score_bucket,age_bucket,nb_loans,default_rate
HIGH,18-25,9624,0.04758935993349958
HIGH,26-40,28540,0.04919411352487737
HIGH,41-60,37880,0.04794086589229145
HIGH,60+,17313,0.04609253162363542
LOW,18-25,391,0.10485933503836317
LOW,26-40,1041,0.138328530259366
LOW,41-60,1412,0.13526912181303116
LOW,60+,689,0.13497822931785197
MEDIUM,18-25,4028,0.08391261171797418
MEDIUM,26-40,11909,0.0848098077084558


### Cross-Analysis — Income × Age

`This analysis examines the combined effect of income level and borrower age on default risk.`\
`It helps identify whether income-related risk patterns vary across different life stages within the portfolio.`


In [0]:
df_income_age = (
    loan_df
        .groupBy("income_bucket", "age_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("income_bucket", "age_bucket")
)

display(df_income_age)


income_bucket,age_bucket,nb_loans,default_rate
HIGH,18-25,8631,0.053064534816359636
HIGH,26-40,25446,0.05450758468914564
HIGH,41-60,34015,0.053300014699397325
HIGH,60+,15378,0.05293276108726753
LOW,18-25,420,0.10952380952380952
LOW,26-40,1419,0.10359408033826638
LOW,41-60,1732,0.10508083140877598
LOW,60+,740,0.10945945945945947
MEDIUM,18-25,4992,0.06670673076923077
MEDIUM,26-40,14625,0.07001709401709402


### Cross-Analysis — Product × Credit Score × Income

`This analysis combines product type, credit score, and income level to provide a multidimensional view of credit risk.`\
`It highlights how product risk interacts with borrower creditworthiness and repayment capacity, allowing a granular identification of high-risk and high-value segments within the portfolio.`


In [0]:
df_prod_score_income = (
    loan_df
        .groupBy("product_type", "credit_score_bucket", "income_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("product_type", "credit_score_bucket", "income_bucket")
)

display(df_prod_score_income)


product_type,credit_score_bucket,income_bucket,nb_loans,default_rate
CONSO,HIGH,HIGH,49421,0.05697982638959147
CONSO,HIGH,LOW,1,0.0
CONSO,HIGH,MEDIUM,16938,0.05791710945802338
CONSO,LOW,LOW,484,0.15702479338842976
CONSO,LOW,MEDIUM,1994,0.15446339017051153
CONSO,MEDIUM,HIGH,9947,0.09882376595958581
CONSO,MEDIUM,LOW,2583,0.12233836624080527
CONSO,MEDIUM,MEDIUM,15121,0.10620990675219893
IMMO,HIGH,HIGH,19353,0.02242546375238981
IMMO,HIGH,MEDIUM,6605,0.02089326267978804


### Analysis — Exposure Buckets

`This analysis segments loans into exposure buckets to examine how credit risk varies with loan size.`\
`It helps assess whether higher exposures concentrate more risk or whether risk remains primarily driven by borrower and product characteristics.`


In [0]:
# bucket creation 

loan_df = loan_df.withColumn(
    "exposure_bucket",
    F.when(F.col("principal_amount") < 5000, "<5K")
     .when((F.col("principal_amount") >= 5000) & (F.col("principal_amount") < 20000), "5K-20K")
     .when((F.col("principal_amount") >= 20000) & (F.col("principal_amount") < 100000), "20K-100K")
     .otherwise("100K+")
)


In [0]:
# analyse
df_exposure_bucket = (
    loan_df
        .groupBy("exposure_bucket")
        .agg(
            F.count("*").alias("nb_loans"),
            F.sum("principal_amount").alias("exposure"),
            F.avg("default_flag").alias("default_rate")
        )
        .orderBy("exposure_bucket")
)

display(df_exposure_bucket)


exposure_bucket,nb_loans,exposure,default_rate
100K+,34989,1.3018866117666883E10,0.028351767698419503
20K-100K,50419,1.7237736303455698E9,0.07096531069636447
5K-20K,41210,5.112893069178891E8,0.07415675806842999
<5K,9306,2.997334845888235E7,0.07833655705996131


In [0]:
loan_df = loan_df.withColumn(
    "vintage_year",
    F.year(F.col("origination_date"))
)
